In [ ]:
from google.colab import drive
import sys
import os

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

drive.mount("/content/gdrive")
#Specify path path of the directory
#path = "/content/gdrive/My Drive/Colab Notebooks/econ_indicator"
#sys.path.append(path)
os.chdir('/content/gdrive/My Drive/Colab Notebooks/econ_indicator')

Mounted at /content/gdrive


In [ ]:
import IPython
from google.colab import output
display(IPython.display.Javascript('''
function ClickConnect(){
  btn=document.querySelector("colab-connect-button")
  if(btn != null){
    console.log("Click colab-connect-button");
    btn.click()
  }
  btn=document.getElementById('ok')
  if(btn != null){
    console.log("Click reconnect");
    btn.click()
  }
}
setInterval(ClickConnect,60000)
'''))

<IPython.core.display.Javascript object>

In [ ]:
# !pwd
!ls

predicted_sales.csv  sales_indicators.csv






*   change epochs, batch size;
*   adding 1/8 layer, change drop out rate
*  change x,y








In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
#df = pd.read_csv('sales_indicators.csv',index_col=0)
df = pd.read_excel('All_Data_Formatted.xlsx', skiprows=1,index_col=0)
df['International_Trade']=df['International_Trade'].div(10**8)
df.reset_index(inplace=True)
df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

FileNotFoundError: ignored

In [ ]:
df['Holiday'] = df['month'].isin([12])
df.Holiday.sum()

In [ ]:
df.replace({False: 0, True: 1}, inplace=True)
df

In [ ]:
df.set_index('date', inplace=True)
df

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
# #break time periods
# #2010-2019 June
# #2019 July-2019 Dec
# df_2010_2019June=df.iloc[84:198,:]
# df_2010_2019June

# df_2010_2018Dec=df.iloc[84:192,:]
# df_2010_2018Dec

df_2010_2019=df.iloc[84:204,:]
df_2010_2019

In [ ]:
# df_2019July_Dec=df.iloc[198:204,:]
# df_2019July_Dec

# df_2019=df.iloc[192:204,:]
# df_2019

In [ ]:
# df.head()

In [ ]:
row_num=df_2010_2019.shape[0]
#row_num#120
120*2/3

In [ ]:
##################################################testing function 
def neural_net(department,econ_id,ep,bat):
  import sys
  Y = df_2010_2019.loc[:,department] 
  print(Y)
  x_temp=['year','month']+econ_id
  X = df_2010_2019.loc[:,x_temp]
  #X = df_2010_2018Dec.loc[:,['year','month']]

  #Split train and test
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 1/10, shuffle=True)

  #Count features for modelization
  X_num_columns= len(X.columns)

  # normalizer=layers.Normalizaton()
  # normalizer.adapt(X_train)

  model = Sequential()
  # model.add(normalizer)
  model.add(Dense(120, activation='relu',input_dim = X_num_columns))
  model.add(Dropout(0.2))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(1,activation='linear'))
  # don't ever put dropout layer in the output layer
  # don't dropout 
  
  #model.add(Dropout(0.2))

  # model.add(Dense(30,
  #               activation='relu'))
  # model.add(Dropout(0.2))

  model.compile(optimizer='adam', loss='mean_squared_error')

  model.fit(X_train, y_train, epochs=ep, batch_size=bat)

  loss = model.evaluate(X_test, y_test)
  # relative error 
  result=(df_2010_2019[department].std()-np.sqrt(loss))/df_2010_2019[department].std()
  return result

In [ ]:
# econ_id=['indust_prod', 'mortgage_rate30']
# x_temp=['year','month']+econ_id
# print(x_temp)

In [ ]:
##########################################testing each department with single economic indicators
# sales=['RSAFS', 'RSFSXMV', 'RSHPCS', 'RSGASS', 'RSCCAS', 'RSSGHBMS', 'RSGMS', 'RSDSELD', 'RSMSR', 'RSNSR', 'RSFSDP', 'RSXFS']
# sales=['RSAFS', 'RSFSXMV']
sales=['RSNSR', 'RSFSDP']
econ_indicators=['indust_prod', 'mortgage_rate30', 'International_Trade', 'CPI','Holiday']

# epoch = [64,512]
batch = [16]
# batch = [16,25,32,64]

# ep = 512
ep = 128
# ep = 64


#creating combinations of economic indicators
comb1 = ['indust_prod', 'mortgage_rate30', 'International_Trade', 'CPI','Holiday']
comb1 = list(map(lambda el:[el], comb1))
# print(len(comb1))

from itertools import combinations
comb2 = list(combinations(econ_indicators, 2))
comb2 = list(map(list, comb2))
# print(len(comb2))

comb3 = list(combinations(econ_indicators, 3))
comb3 = list(map(list, comb3))
# print(len(comb3))

comb4 = list(combinations(econ_indicators, 4))
comb4 = list(map(list, comb2))
# print(len(comb4))

comb5 = [['indust_prod', 'mortgage_rate30', 'International_Trade', 'CPI','Holiday']]
# print(len(comb5))

comb_econ_indicators = comb1+comb2
# comb_econ_indicators = comb1+comb2+comb3+comb4+comb5

# def neural_net(department,econ_id,ep,bat):
#   return 1

# print(len(sales))#12
# print(len(comb_econ_indicators))#36

import numpy as np
loss = np.zeros((len(batch), len(sales), len(comb_econ_indicators)))
#print(loss)
# for department,econ_id in zip(sales, comb_econ_indicators):
#   temp = [neural_net(department,econ_id,ep,bat) for bat in batch]
#   loss.append(temp)


for i, bat in enumerate(batch):
  for j, department in enumerate(sales):
      for k, econ_id in enumerate(comb_econ_indicators):
        temp = neural_net(department,econ_id,ep,bat)
        loss[i,j,k]=temp




#department='RSAFS'
# econ_id=['indust_prod', 'mortgage_rate30']
# ep = 512
# bat = 32
#neural_net(department,econ_id,ep,bat)

# sales_indust = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_mortgage = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_intern = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_cpi = [neural_net(i,econ_id,ep,bat) for i in sales]



In [ ]:
# np.sum(loss)
# loss.shape
print(loss)
print(loss.shape)

In [ ]:
df_loss=pd.DataFrame(data = [[x] for x in loss])
df_loss.to_csv("loss.csv")